In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score
import transformers
import torch
#device = 'cpu' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(device)
#print(device)
from torch import nn
import torch.nn.functional as F
#from tqdm import trange
from tqdm.notebook import tqdm, trange
import os
#from torch.optim import AdamW
from transformers import AdamW, get_linear_schedule_with_warmup
from model import TransTCN

cpu


In [2]:
rev = pd.read_json('yelp_review_training_dataset.jsonl',lines=True)

In [3]:
def clean_text(text_data):
    text_data = text_data.lower()
    #text_data = re.sub('\w*\d\w*', "", text_data)
    return text_data  

In [4]:
rev['clean_text'] = rev.text.apply(lambda x: clean_text(x))

In [5]:
X = rev['clean_text']
y = rev['stars']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.4, random_state=123)

In [6]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
class ProcessData(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_len, labels):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.labels = labels
    
    def __getitem__(self, index):
        review = self.data[index]
        label = self.labels[index]
        return self.tokenizer.encode_plus(review, max_length=self.max_len, padding='max_length', return_attention_mask=True, return_tensors='pt', truncation=True), review, torch.tensor([label-1]).to(torch.long)
    
    def __len__(self):
        review_length = len(self.data)
        return review_length

In [8]:
tokenized_training_data = ProcessData(X_train[:100].to_numpy().tolist(), tokenizer, 150, y_train[:100].to_numpy())
tokenized_validation_data = ProcessData(X_val[:100].to_numpy().tolist(), tokenizer, 150, y_val[:100].to_numpy())
tokenized_test_data = ProcessData(X_test[:100].to_numpy().tolist(), tokenizer, 150, y_test[:100].to_numpy())

In [9]:
#Create dataloader
params = {'batch_size': 32,
          'num_workers': 0}
loader_tokenized_training_data = torch.utils.data.DataLoader(tokenized_training_data, **params)
loader_tokenized_validation_data = torch.utils.data.DataLoader(tokenized_validation_data, **params)
loader_tokenized_test_data = torch.utils.data.DataLoader(tokenized_test_data, **params)

In [10]:
#classes, input_size, num_channels, kernel_size
model = TransTCN(5, 1, [1,1], kernel_size=2)
#model.to(device)

In [11]:
from torch.optim import Adam
criterion = nn.CrossEntropyLoss().to(device)
optimizer = AdamW(model.parameters(), lr=5e-5, correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,num_training_steps=len(loader_tokenized_training_data) * 5)

In [12]:
def training(model, data_loader, size):
    model = model.train()
    losses = []
    training_acc = 0
    for data in tqdm(data_loader):
        batch_ids = data[0]['input_ids']
        batch_ids = batch_ids.flatten().reshape((batch_ids.shape[0], batch_ids.shape[2]))
        batch_masks = data[0]['attention_mask']
        batch_masks = batch_masks.flatten().reshape((batch_masks.shape[0], batch_masks.shape[2]))
        data[2] = data[2].to(device)
       
       
        output = model(batch_ids.to(device), batch_masks.to(device))
        #print(output)
     
        prediction = torch.max(output, 1)[1]
 
        training_loss = criterion(output, torch.flatten(data[2]))
        training_acc += torch.sum(prediction == torch.flatten(data[2]))
   
        losses.append(training_loss.item())
        training_loss.backward()
        #nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
    return training_acc / size, np.mean(losses)

In [13]:
def evaluate(model, data_loader, size):
    model = model.eval()
    losses = []
    validation_acc = 0
    with torch.no_grad():
        for vdata in tqdm(data_loader):
            vbatch_ids = vdata[0]['input_ids']
            vbatch_ids = vbatch_ids.flatten().reshape((vbatch_ids.shape[0], vbatch_ids.shape[2]))
            vbatch_masks = vdata[0]['attention_mask']
            vbatch_masks = vbatch_masks.flatten().reshape((vbatch_masks.shape[0], vbatch_masks.shape[2]))
            vdata[2] = vdata[2].to(device)
           

            voutput = model(vbatch_ids.to(device), vbatch_masks.to(device))
            vprediction = torch.max(voutput, 1)[1]
            
            vloss = criterion(voutput, torch.flatten(vdata[2]))
            validation_acc += torch.sum(vprediction == torch.flatten(vdata[2]))
            losses.append(vloss.item())
    return validation_acc / size, np.mean(losses)

In [ ]:
highest_acc = 0
for epoch in trange(1):
    print('Epoch: ' , str(epoch))
    print('==================================')
    training_accuracy, training_loss = training(model,loader_tokenized_training_data,len(X_train[:100]) )
    validation_accuracy, validation_loss = evaluate(model, loader_tokenized_validation_data, len(X_val[:100]))
    
    print('Training accuracy: ', training_accuracy )
    print('Training loss: ', training_loss)
    print('Validation accuracy: ', validation_accuracy)
    print('Validation loss: ', validation_loss)

Epoch:  0


0
